In [ ]:
import requests
import json
import pandas as pd
import pyasn

In [ ]:
pauls_key = "32ea6dea-ec86-4ed9-9f20-004088064867"

In [ ]:
def get_traceroute_data(api_key):
    """
    Function filters traceroutes out of all measurements and returns only valid ones tagged with 'orangecat'.
    @params
        api_key: api_key from ripe atlas with 'List your measurements' permission.
    @return
        pandas dataframe with traceroute data
    """
    measurements = requests.get(f"https://atlas.ripe.net/api/v2/measurements/my/?key={api_key}").json()['results']
    
    tracerout_list = []

    for measurement in measurements:
        if "traceroute" in measurement['tags'] and "orangecat" in measurement['tags']:
            result_url = measurement['result']
            start_time = measurement['start_time']
            tracerout_url = f"{result_url}?start={start_time}"

            traceroute_response = requests.get(tracerout_url)
            if traceroute_response.status_code == 200:
                tracerout_list.extend(traceroute_response.json())

    return pd.DataFrame(tracerout_list)

In [ ]:
df_traceroutes = get_traceroute_data(pauls_key)

In [ ]:
mapping = pd.read_csv("data/mapping.csv", index_col=0)    # This is the mapping from previous notebook that I stored as csv

In [ ]:
def get_asn(ip):
    asndb = pyasn.pyasn('data/ipasn_20230214.dat')
    return ansdb.lookup(ip)

def get_asn_path(traceroute):
    """Gets the AS numbers to a tracerout. Only considers the first ip alternative of each hop."""
    ips = []
    asn_path = []

    for hop in traceroute:
        try:
            ip = hop['result'][0]['from']
            if not ips or ips[-1] != ip:
                ips.append(ip)
                asn = get_asn(ip)
                if not asn_path or asn_path[-1] != asn_object:
                    asn_path.append(asn_object)
        except Exception:
            pass
    return asn_path

In [ ]:
df_traceroutes['asn_path'] = df_traceroutes['result'].apply(get_asn_path)